In [ ]:
# -------------------------------
# 0. 라이브러리 및 환경 설정
# -------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from scipy.stats import zscore, pointbiserialr
from sklearn.preprocessing import PowerTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings("ignore")

# 그래프 한글폰트(선택) ─ macOS 예시
plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'Apple SD Gothic Neo' # 띄어쓰기 포함
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

In [ ]:
kospi = pd.read_csv("../data/raw/코스피_상장폐지.csv")
kosdaq = pd.read_csv("../data/raw/코스닥_상장폐지.csv")

In [ ]:
fail = pd.concat([kospi, kosdaq])

In [ ]:
fail.to_csv("../data/processed/fail.csv")

In [ ]:
flag = fail['종목코드'].unique()

bs = pd.read_csv("../data/processed/BS.csv")

bsset = set(bs['거래소코드'].unique())
flagset = set(flag)

print(f"BS에 있는 기업수: {len(bsset)}")
print(f"flag에 있는 기업수: {len(flagset)}")
print(f"BS에 있는 상장폐지 기업수: {len(bsset & flagset)}")
print(f"BS에 없는 상장폐지 기업수: {len(flagset - bsset)}")
print(f"최종 기업수: {len(bsset - flagset)}")


In [ ]:
c_ratio = pd.read_csv("../data/raw/연결 재무비율(IFRS).csv")
n_ratio = pd.read_csv("../data/raw/재무비율(IFRS).csv")
c_bs = pd.read_csv("../data/raw/연결 재무제표(IFRS).csv")
n_bs = pd.read_csv("../data/raw/재무제표(IFRS).csv")
delisting = pd.concat([kospi, kosdaq])
df = pd.DataFrame()

In [ ]:
c_bs.columns.tolist()

In [ ]:
n_bs.columns.tolist()

In [ ]:
bs = pd.read_csv("../data/processed/BS.csv")
ratio = pd.read_csv("../data/processed/ratio.csv")

print(bs.shape)
print(ratio.shape)

df = pd.concat([bs, ratio], axis=1)
df.to_csv("../data/processed/bs_ratio.csv", index=False)

In [ ]:
df.columns

In [ ]:
df.columns.tolist()

In [ ]:
# -------------------------------
# 1. 데이터 불러오기
# -------------------------------
### TODO: 실제 파일 경로를 지정하세요
df = pd.read_csv("../data/processed/bs_ratio.csv")

# -------------------------------
# 2. 주요 변수 지정
# -------------------------------
### TODO: 컬럼명을 실제 데이터에 맞춰 수정
id_col      = '거래소코드'
keep_cols = [
    '[A100000000]자산(*)(IFRS연결)(천원)',                                    # 총자산
    '[A800000000]부채(*)(IFRS연결)(천원)',                                    # 총부채
    '[A600000000]자본(*)(IFRS연결)(천원)',                                    # 자본
    '[B420000000]* (정상)영업손익(보고서기재)(3개월)(IFRS연결)(천원)',     # 영업이익 (분기)
    '[B840000000]당기순이익(손실)(3개월)(IFRS연결)(천원)',                   # 순이익 (분기)
    '[D100000000]영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)',    # 영업현금흐름
    '부채비율(IFRS연결)',                                                      # Debt Ratio
    '차입금의존도(IFRS연결)',                                                  # Borrowing Dependence
    '총자본회전률(IFRS연결)',                                                  # Asset Turnover
    'PBR(최저)(IFRS연결)'                                                      # PBR (Lowest)
]
df = df.drop(columns=[c for c in df.columns if c not in keep_cols])
df.columns = ['Asset', 'Debt', 'Equity', 'OperatingIncome', 'NetIncome', 'OperatingCashFlow', 'DebtRatio', 'BorrowingDependence', 'AssetTurnover', 'PBR']
cat_cols = []                  # 범주형 변수 있다면 추가
df.columns

In [ ]:
# 2-1. 정보 요약
display(df.info())
display(df.head())

# # 2-2. 중복 확인 (동일 기업·연도 중복행 탐지)
# dup_mask = df.duplicated(subset=[id_col, 'year'])
# print(f"중복 행 수: {dup_mask.sum()}")
# if dup_mask.any():
#     df = df[~dup_mask]  # 필요 시 제거

In [ ]:
# 3-1. 변수별 결측 비율
null_ratio = df.isna().mean().sort_values(ascending=False)
display(null_ratio.head(10))

# 3-2. 히트맵 시각화
plt.figure(figsize=(10,4))
msno.matrix(df.sample(min(1000, len(df_bs))))
plt.show()

In [ ]:
# 4-1. 기술통계
desc = df.describe().T
display(desc)

# 4-2. z-score 기반 이상치 마스크
z_scores = df.apply(zscore)
outlier_mask = (np.abs(z_scores) > 3).any(axis=1)
print(f"z>|3| 이상치 행 비율: {outlier_mask.mean():.2%}")

# 4-3. 상자그림(샘플 10개 변수)
numeric_cols = df.select_dtypes(include='number').columns

for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col], orient='h')
    plt.title(f"Boxplot of {col}")
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

In [ ]:
# 6-1. 피어슨 상관행렬
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Matrix"); plt.show()

# # 6-2. VIF 계산
# X = df.drop(columns=[], errors='ignore')  # 필요 시 제외 변수 지정
# vif_vals = pd.Series(
#     [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
#     index=X.columns, name='VIF'
# )
# display(vif_vals.sort_values(ascending=False).head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# ─────────────────────────────────────────────
# QQ-Plot: 각 변수별로 화면에 표시
# ─────────────────────────────────────────────
for col in df.columns:
    sm.qqplot(df[col].dropna(), line='45', fit=True)
    plt.title(f"QQ Plot of {col}")
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel("Sample Quantiles")
    plt.tight_layout()
    plt.show()
    plt.close()

# ─────────────────────────────────────────────
# Scatter: 'Asset' 기준으로 나머지 변수들과 화면에 표시
# ─────────────────────────────────────────────
for col in df.columns:
    if col == 'Asset':
        continue
    plt.figure(figsize=(6, 6))
    sns.scatterplot(x=df['Asset'], y=df[col], alpha=0.6)
    plt.title(f"Asset vs {col}")
    plt.xlabel("Asset")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()
    plt.close()


In [ ]:
industry = pd.read_excel("../data/raw/산업분류.xlsx")

In [ ]:
industry["통계청 한국표준산업분류 11차(대분류)"].value_counts()

In [ ]:
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "금융 및 보험업"]
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "부동산업"]
industry = industry[industry['통계청 한국표준산업분류 11차(대분류)'] != "전기, 가스, 증기 및 공기조절 공급업 (공공 유틸리티)"]
industry

In [ ]:
len(c_bs)

In [ ]:
# industry에 존재하는 거래소코드 목록 추출
valid_codes = industry['거래소코드'].unique()

# 각 데이터프레임에서 valid_codes에 포함된 행만 남기기
c_bs     = c_bs    [c_bs    ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
c_ratio  = c_ratio [c_ratio ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
n_bs     = n_bs    [n_bs    ['거래소코드'].isin(valid_codes)].reset_index(drop=True)
n_ratio  = n_ratio [n_ratio ['거래소코드'].isin(valid_codes)].reset_index(drop=True)

In [ ]:
c_bs.to_csv("../data/raw/연결재무제표.csv", index=False)
n_bs.to_csv("../data/raw/개별재무제표.csv", index=False)
c_ratio.to_csv("../data/raw/연결재무비율.csv", index=False)
n_ratio.to_csv("../data/raw/개별재무비율.csv", index=False)